<a href="https://colab.research.google.com/github/NicolasSpettel/lab-neural-networks/blob/master/your-code/challenge_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [39]:
import pandas as pd
df = pd.read_csv("tic-tac-toe.csv")

In [40]:
df.describe()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
count,958,958,958,958,958,958,958,958,958,958
unique,3,3,3,3,3,3,3,3,3,2
top,x,x,x,x,x,x,x,x,x,True
freq,418,378,418,378,458,378,418,378,418,626


In [41]:
df.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [42]:
ttt_map = {
 "b" : "0",
 "x" : "1",
 "o" : "-1"}

df = df.replace(ttt_map).astype(int)

In [43]:
features = df.iloc[:, :-1] # All columns except the last one
labels = df.iloc[:, -1]    # The last column

In [44]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Scale the features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Split the training and test data
x_train, x_test, y_train, y_test = train_test_split(features_scaled, labels, test_size=0.2, random_state=42)

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [45]:
import tensorflow.keras
from keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


num_features = 9
num_classes = 2

# batch_size = 64
epochs = 250 # Increased the number of epochs to train for longer

# define classification model
def classification_model():
    # create model
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(num_features,)))
    model.add(Dropout(0.4))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(num_classes, activation='softmax'))

    model.summary()

    # Adjusted the optimizer to Adam and set a custom learning rate
    my_opt = tf.keras.optimizers.Adam(learning_rate=0.0005)

    # compile model
    model.compile(optimizer=my_opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

model = classification_model()

checkpoint_callback = ModelCheckpoint(
    filepath='best_model.keras',
    monitor='val_loss', # Monitor the validation loss
    verbose=1,
    save_best_only=True, # Save only the best model found so far
    mode='min' # 'min' because we want to minimize the loss
)

early_stopping_callback = EarlyStopping(
    monitor='val_loss', # Monitor the validation loss
    patience=20, # Number of epochs with no improvement after which training will be stopped
    verbose=1,
    mode='min',
    restore_best_weights=True # Restore model weights from the epoch with the best value of the monitored quantity
)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_60 (Dense)                │ (None, 128)            │         1,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_44 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_61 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_45 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_62 (Dense)                │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,666 (37.76 KB)

 Trainable params: 9,666 (37.76 KB)

 Non-trainable params: 0 (0.00 B)

In [46]:
model = classification_model()
history = model.fit(x_train, y_train,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=[checkpoint_callback,
                    early_stopping_callback]
                    )
model.load_weights('best_model.keras')

Model: "sequential_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_63 (Dense)                │ (None, 128)            │         1,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_46 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_64 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_47 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_65 (Dense)                │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,666 (37.76 KB)

 Trainable params: 9,666 (37.76 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/250
14/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5731 - loss: 0.7003
Epoch 1: val_loss improved from inf to 0.60481, saving model to best_model.keras
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.5793 - loss: 0.6953 - val_accuracy: 0.6667 - val_loss: 0.6048
Epoch 2/250
23/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6194 - loss: 0.6512   
Epoch 2: val_loss improved from 0.60481 to 0.56788, saving model to best_model.keras
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6211 - loss: 0.6505 - val_accuracy: 0.7292 - val_loss: 0.5679
Epoch 3/250
21/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6892 - loss: 0.5754 
Epoch 3: val_loss improved from 0.56788 to 0.54295, saving model to best_model.keras
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6875 - loss: 0.5782 - val_accuracy: 0.7500 - val_loss: 0.5430
Epoch 4/250
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6929 - loss: 0.6005 
Epoch 4: val_loss improved from 0.54295 to 0.52762, saving

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [47]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.05215884745121002
Test accuracy: 0.9791666865348816


In [48]:
import numpy as np

# Choose 5 random indices from the test dataset
num_predictions_to_check = 5
random_indices = np.random.choice(len(x_test), num_predictions_to_check, replace=False) # Use replace=False to get unique indices

# Get the corresponding random samples from the test data
random_x_test = x_test[random_indices]
random_y_test = y_test.iloc[random_indices]


# Use the loaded model to predict the classes of the random samples
predictions = model.predict(random_x_test)

# The output of model.predict() with softmax is a set of probabilities for each class.
# We need to find the class with the highest probability.
predicted_classes = np.argmax(predictions, axis=1)

# Now, compare the predicted classes with the actual labels
print("\n--- Predictions vs. Actual Labels ---")
for i in range(num_predictions_to_check):
    actual_label = random_y_test.iloc[i]
    predicted_label = predicted_classes[i]

    print(f"Sample {i+1}:")
    print(f"  Input Board State: {random_x_test[i]}") # Use .values to display the numpy array
    print(f"  Predicted Label:   {predicted_label}")
    print(f"  Actual Label:      {actual_label}")
    print(f"  Prediction Correct: {predicted_label == actual_label}")
    print("-" * 20)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step

--- Predictions vs. Actual Labels ---
Sample 1:
  Input Board State: [ 1.03516957 -1.2235944   1.03516957 -0.05838224  0.96958654 -1.2235944
  1.03516957 -1.2235944  -0.09819323]
  Predicted Label:   1
  Actual Label:      1
  Prediction Correct: True
--------------------
Sample 2:
  Input Board State: [-0.09819323  1.10682993  1.03516957 -1.2235944  -1.24199419 -1.2235944
  1.03516957 -0.05838224 -0.09819323]
  Predicted Label:   0
  Actual Label:      0
  Prediction Correct: True
--------------------
Sample 3:
  Input Board State: [ 1.03516957 -0.05838224  1.03516957 -1.2235944  -0.13620382  1.10682993
 -1.23155602 -1.2235944   1.03516957]
  Predicted Label:   1
  Actual Label:      1
  Prediction Correct: True
--------------------
Sample 4:
  Input Board State: [ 1.03516957 -1.2235944   1.03516957  1.10682993 -0.13620382 -0.05838224
  1.03516957 -1.2235944  -1.23155602]
  Predicted Label:   1
  Actual Label:      1
  Prediction Correct: True
--

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [49]:
# added another layer and the adam optimizer and increased epochs, accuracy and loss are better but still not <0.1 and >0.9

**Which approach(es) did you find helpful to improve your model performance?**

In [50]:
# Increasing the amount of epochs seemed very helpful.
# Adding more layers alone didn't seem to help a lot.
# Increasing the layer units seemed effective.
# Decreasing the learning_rate helped to find the best model
# Increasing dropout rate to reduce overfitting
# In general turning all "knobs" up seems to increase performance for this one